In [2]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init
import time
import random
import pandas as pd
import openpyxl
import logging
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [3]:
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [4]:
# Define log file path in Google Drive
log_file = "/content/drive/MyDrive/NNOutputs/CFD.log"

# Create a custom logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Clear previous handlers if re-running in Colab
if logger.hasHandlers():
    logger.handlers.clear()

# Create file handler
file_handler = logging.FileHandler(log_file, mode='w')
file_handler.setFormatter(
    logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s',
                      datefmt='%Y-%m-%d %H:%M:%S')
    )

# Add handler to logger
logger.addHandler(file_handler)

# Example usage
logger.info("Logging setup complete.")

In [ ]:
#### material params:- RT-35, aluminum ####
k_therm = 0.2
L = 160000
Cp = 2000
rho = 800

T_solidus = 305
T_liquidus = 311

#### Heat source ####
s = 5000

#### Normalising coeffs/ actual domain and temporal simulation params ####
T_change = 0.2
delta_x = 0.03
delta_y = 0.03
delta_t = 4000
T_sc = (T_liquidus - T_solidus)/T_change

#### Normalised Coefficients  ####
k1 = k_therm/(rho*Cp*delta_x**2)*delta_t
k2 = k_therm/(rho*(Cp+L/(T_liquidus - T_solidus))*delta_x**2)*delta_t
s_norm = s*delta_t/(rho*Cp*T_sc)

#### FVM Domain Parameters ####
N_x = 81
N_y = 81
x_l = 0
y_b = 0
x_r = 1
y_t = 1
N_t = 300000
dt = 0.00001

#### Circular heat source ####
x_c = 0.5
y_c = 0.5

# X, Y matrix
X = np.linspace(x_l,x_r,N_x)
dx = X[1] - X[0]
X = np.tile(X,N_y)
Y = np.linspace(y_b,y_t,N_y)
dy = Y[1] - Y[0]
Y = np.repeat(Y,N_x)
X = np.reshape(X, (N_y,N_x))
Y = np.reshape(Y, (N_y,N_x))
dist = np.sqrt(( X - x_c )**2 + ( Y - y_c )**2)

# liquid fraction, heat source and initial temperature matrix
f = np.zeros((N_y,N_x))
T = np.zeros((N_y,N_x))

# cfd check
k1*dt/dx**2, k2*dt/dx**2, s_norm, N_t*dt

In [8]:
aN = np.zeros((N_x,N_y))
aS = np.zeros((N_x,N_y))
aE = np.zeros((N_x,N_y))
aW = np.zeros((N_x,N_y))
aP = np.zeros((N_x,N_y))
flag = np.zeros((N_x,N_y))

for i in range(N_x):
    for j in range(N_y):

        if Y[i, j]!=y_t:
            aN[i, j] = k2*dt/dy**2

        if Y[i, j]!=y_b:
            aS[i, j] = k2*dt/dy**2

        if X[i, j]!=x_l:
            aW[i, j] = k2*dt/dx**2

        if X[i, j]!=x_r:
            aE[i, j] = k2*dt/dx**2

        aP[i, j] = 1 - (aN[i, j] + aS[i, j] + aW[i, j] + aE[i, j])

In [9]:
storage_interval = 10000

T_str = []
T_str.append(np.copy(T))
T_new = np.copy(T)

f_str = []
f_str.append(np.copy(f))
f_new = np.copy(f)

# with pd.ExcelWriter("resultsradialsource.xlsx", engine="openpyxl") as writer:
spreadsheet = gc.create('resultsradialsource_5k')

for t in range(N_t):

    if t%100 == 0:
        t_val = (t+1) * dt
        center_temp = T_new[int(N_y/2), int(N_x/2)]
        edge_temp = T_new[int(N_y/2), 0]
        logging.info(f"iteration = {t}, t = {t_val:.6f}, cfd center temp = {center_temp:.6e}, edge temp at centerline = {edge_temp:.6e}")

    T = np.copy(T_new)
    f = np.copy(f_new)

    # Temperature Update
    for i in range(N_y):
        for j in range(N_x):

            if ( X[i, j]!=x_l and X[i, j]!=x_r and Y[i, j]!=y_t and Y[i, j]!=y_b ):

                T_new[i, j] = ( aN[i, j]*T[i+1][j] + aS[i, j]*T[i-1][j] +
                                aW[i, j]*T[i][j-1] + aE[i, j]*T[i][j+1] + aP[i, j]*T[i, j] + s_norm*(1-dist[i, j]/0.71)*dt)

                f_new[i, j] = T_new[i, j]/T_change

    # LEFT BOUNDARY
    for i in range(1, N_y - 1):
        j=0
        T_new[i, j] = T_new[i, j+1]
        f_new[i, j] = (T_new[i, j] /T_change)

    # RIGHT BOUNDARY
    for i in range(1, N_y - 1):
        j=N_x-1
        T_new[i, j] = T_new[i, j-1]
        f_new[i, j] = (T_new[i, j] /T_change)

    # BOTTOM BOUNDARY
    for j in range(1, N_x - 1):
        i=0
        T_new[i, j] = T_new[i+1, j]
        f_new[i, j] = (T_new[i, j] /T_change)

    # TOP BOUNDARY
    for j in range(1, N_x - 1):
        i=N_y-1
        T_new[i, j]  = T_new[i-1, j]
        f_new[i, j] = (T_new[i, j] /T_change)

    # CORNER 1
    i=0
    j=0
    T_new[i, j]  = (T_new[i+1, j]  + T_new[i, j+1])/2
    f_new[i, j]  = (T_new[i, j] /T_change)

    # CORNER 2
    i=0
    j=N_x-1
    T_new[i, j]  = (T_new[i+1, j]  + T_new[i, j-1])/2
    f_new[i, j]  = (T_new[i, j] /T_change)

    # CORNER 3
    i=N_y-1
    j=0
    T_new[i, j]  = (T_new[i-1, j] + T_new[i, j+1])/2
    f_new[i, j]  = (T_new[i, j] /T_change)

    # CORNER 4
    i=N_y-1
    j=N_x-1
    T_new[i, j]  = (T_new[i-1, j] + T_new[i, j-1])/2
    f_new[i, j]  = (T_new[i, j] /T_change)


######### Liquid Fraction Update #########
    for i in range(N_y):
        for j in range(N_x):

            if T_new[i, j]>=T_change or f_new[i, j]>=1:

                if Y[i, j]!=y_t:
                    aN[i, j] = k1*dt/dy**2

                if Y[i, j]!=y_b:
                    aS[i, j] = k1*dt/dy**2

                if X[i, j]!=x_l:
                    aW[i, j] = k1*dt/dx**2

                if X[i, j]!=x_r:
                    aE[i, j] = k1*dt/dx**2

                aP[i, j] = 1 - (aN[i, j] + aS[i, j] + aW[i, j] + aE[i, j])

    if (t+1)%500==0 or t==0:
        T_str.append(np.copy(T_new))
        f_str.append(np.copy(f_new))
        # df = pd.DataFrame(T_new)
        sheet_name = f"t_{(t+1)*dt}"
        try:
            worksheet = spreadsheet.worksheet(sheet_name)
        except gspread.WorksheetNotFound:
            worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=N_y, cols=N_x)
        worksheet.update([row.tolist() for row in T_new])
        # df.to_excel(writer, sheet_name=sheet_name, index=False)
        t_val = (t+1) * dt
        center_temp = T_new[int(N_y/2), int(N_x/2)]
        edge_temp = T_new[int(N_y/2),0]
        logging.info(f"saved. t = {t_val:.6f}, center temp = {center_temp:.6e}, edge temp at centerline = {edge_temp:.6e}")
